In [1]:
%%time
import sys
import re
import math
import requests
import enlighten
import sqlite3

import numpy as np
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)

url = 'http://200.152.38.155/CNPJ/'
formato_data = '%Y%m%d'
PATH_BD = 'data/CNPJ_full.db'

CPU times: total: 78.1 ms
Wall time: 733 ms


In [2]:
%%time
list_urls = re.findall(r'href=[\'"]?([^\'" >]+)', requests.get(url).text)
filtered = list(filter(lambda url: url.split(".")[-1] in ["zip"], list_urls))
tamanho = len(filtered)

CPU times: total: 0 ns
Wall time: 164 ms


In [ ]:
%%time
MANAGER = enlighten.get_manager()
bara_lista = MANAGER.counter(total=tamanho, desc='Arquivos', unit='Arquivos')

print("")
print(f"Fazendo download dos {tamanho} arquivos encontrados.", flush=False)
print("")


for indice, elemento in enumerate(filtered):
    bara_lista.update()
    
    r = requests.get(url + elemento, stream=True, headers={"Accept-Encoding": None})
    if r.status_code == requests.codes.ok:
        assert r.status_code == 200, r.status_code
        dlen = int(r.headers.get('Content-Length', '0')) or None
        barra_download = MANAGER.counter(color = 'green', desc= f'Arquivo {indice + 1} de {tamanho}', 
                             total = dlen and math.ceil(dlen / 2 ** 5), 
                             unit = 'MiB', leave = False)
        with barra_download as ctr, \
             open("zips/" + elemento, 'wb', buffering = 2 ** 20) as f:
            for chunk in r.iter_content(chunk_size = 2 ** 20):
                # print(chunk[-16:].hex().upper())
                print(f'\r{elemento} ------ Está sendo baixado!!!', end='', flush=True)
                sys.stdout.flush()
                f.write(chunk)
                ctr.update()
        
    else:
        r.raise_for_status()
print(f'\r Download Concluido!!!', end='', flush=True)
MANAGER.stop()


Fazendo download dos 37 arquivos encontrados.

K3241.K03200Y1.D20514.ESTABELE.zip ------ Está sendo baixado!!!

In [ ]:
%%time
dict_aux = {
    'SIMPLES': [
        'DADOS_SIMPLES',
        'CREATE TABLE IF NOT EXISTS DADOS_SIMPLES (\n  CNPJ               VARCHAR (8) PRIMARY KEY,\n    OPCAO_SIMPLES      VARCHAR (1),\n    DATA_OPCAO_SIMPLES DATE,\n    DATA_EXCLUSAO      DATE,\n    OPCAO_MEI          VARCHAR (1),\n    DATA_OPCAO_MEI     DATE,\n    DATA_EXCLUSAO_MEI  DATE\n)',
        ['cnpj','opcao_simples','data_opcao_simples','data_exclusao','opcao_mei','data_opcao_mei','data_exclusao_mei'],
        {'cnpj': str}
        ],
    'CNAECSV': [
        'CNAES',
        'CREATE TABLE IF NOT EXISTS CNAES (\n    CODIGO    VARCHAR PRIMARY KEY,\n    DESCRICAO VARCHAR\n)',
        ['codigo','descricao'],
        {'descricao': str}
        ],
     'MOTICSV': [
        'MOTIVOS',
        'CREATE TABLE IF NOT EXISTS MOTIVOS (\n    CODIGO    INTEGER PRIMARY KEY,\n    DESCRICAO VARCHAR\n)',
        ['codigo','descricao'],
        {'descricao': str}
        ],
     'MUNICCSV': [
        'MUNICIPIOS',
        'CREATE TABLE IF NOT EXISTS MUNICIPIOS (\n    CODIGO    INTEGERL PRIMARY KEY,\n    DESCRICAO VARCHAR\n)',
        ['codigo','descricao'],
        {'descricao': str}
        ],
     'NATJUCSV': [
        'NATUREZAS',
        'CREATE TABLE IF NOT EXISTS NATUREZAS (\n    CODIGO    INTEGER PRIMARY KEY,\n    DESCRICAO VARCHAR\n)',
        ['codigo','descricao'],
        {'descricao': str}
        ],
     'PAISCSV': [
        'PAISES',
        'CREATE TABLE IF NOT EXISTS PAISES (\n    CODIGO    INTEGER PRIMARY KEY,\n    DESCRICAO VARCHAR\n)',
        ['codigo','descricao'],
        {'descricao': str}
        ],
     'QUALSCSV': [
        'QUALIFICACOES',
        'CREATE TABLE IF NOT EXISTS QUALIFICACOES (\n    CODIGO    INTEGER PRIMARY KEY,\n    DESCRICAO VARCHAR\n)',
        ['codigo','descricao'],
        {'descricao': str}
        ],
     'EMPRECSV': [
        'EMPRESAS',
        'CREATE TABLE IF NOT EXISTS EMPRESAS (\n    CNPJ            VARCHAR (8) PRIMARY KEY,\n    RAZAO           VARCHAR,\n    NATUREZA        INTEGER,\n    QUALIFICACAO_PF INTEGER,\n    CAPITAL         FLOAT,\n    PORTE           INTEGER,\n    ENTE_FEDERATIVO INTEGER\n)',
        ['cnpj','razao','natureza','qualificacao_pf','capital','porte','ente_federativo'],
        {'cnpj': str}
        ],
    'ESTABELE': [
        'ESTABELECIMENTOS',
        'CREATE TABLE IF NOT EXISTS ESTABELECIMENTOS \n                    (CNPJ                   VARCHAR (8),\n                    CNPJ_ORDEM             VARCHAR (4),\n                    CNPJ_DV                VARCHAR (2),\n                    MATRIZ_FILIAL          INTEGER,\n                    NOME                   VARCHAR,\n                    SITUACAO               INTEGER,\n                    DATA_SITUACAO          DATE,\n                    MOTIVO_SITUACAO        VARCHAR,\n                    CIDADE_EXTERIOR        VARCHAR,\n                    PAIS                   INTEGER,\n                    INICIO_ATIVIDADE       DATE,\n                    CNAE_FISCAL            VARCHAR,\n                    CNAE_SECUNDARIO        VARCHAR,\n                    TIPO_LOGRADOURO        VARCHAR,\n                    LOGRADOURO             VARCHAR,\n                    NUMERO                 VARCHAR,\n                    COMPLEMENTO            VARCHAR,\n                    BAIRRO                 VARCHAR,\n                    CEP                    VARCHAR (8),\n                    UF                     VARCHAR (2),\n                    MUNICIPIO              INTEGER,\n                    DDD_1                  VARCHAR,\n                    TELEFONE_1             VARCHAR (11),\n                    DDD_2                  INTEGER,\n                    TELEFONE_2             VARCHAR (11),\n                    DDD_FAX                INTEGER,\n                    NUMERO_FAX             VARCHAR (11),\n                    EMAIL                  VARCHAR,\n                    SITUACAO_ESPECIAL      VARCHAR,\n                    DATA_SITUACAO_ESPECIAL DATE \n                    )',
        ['cnpj','cnpj_ordem','cnpj_dv','matriz_filial','nome','situacao','data_situacao','motivo_situacao','cidade_exterior','pais','inicio_atividade','cnae_fiscal','cnae_secundario','tipo_logradouro','logradouro','numero','complemento','bairro','cep','uf','municipio','ddd_1','telefone_1','ddd_2','telefone_2','ddd_fax','numero_fax','email','situacao_especial','data_situacao_especial'],
        {'cnpj': str, 'cnpj_ordem': str, 'cnpj_dv': str, 'MOTIVO_SITUACAO': int}
        ],
    'SOCIOCSV': [
        'SOCIOS',
        'CREATE TABLE IF NOT EXISTS SOCIOS (\n    CNPJ                             VARCHAR (8) ,\n    IDENTIFICADOR_SOCIO              INTEGER,\n    NOME                             VARCHAR,\n    CPF_CNPJ                         VARCHAR (14),\n    QUALIFICACAO                     INTEGER,\n    DATA_ENTRADA_SOCIEDADE           DATE,\n    CODIGO_PAIS                      INTEGER,\n    CPF_REPRESENTANTE_LEGAL          VARCHAR (11),\n    NOME_REPRESENTANTE_LEGAL         VARCHAR,\n    QUALIFICACAO_REPRESENTANTE_LEGAL INTEGER,\n    FAIXA_ETARIA                     INTEGER\n)',
        ['cnpj','identificador_socio','nome','cpf_cnpj','qualificacao','data_entrada_sociedade','codigo_pais','cpf_representante_legal','nome_representante_legal','qualificacao_representante_legal','faixa_etaria'],
        {'cnpj': str}
        ]    
    }

In [ ]:
%%time
connection = sqlite3.connect(PATH_BD)
con = connection.cursor()

In [ ]:
%%time
manager = enlighten.get_manager()
status_bar = manager.status_bar('Convertendo...',
                            color='blue',
                            justify=enlighten.Justify.CENTER)
pbar = enlighten.Counter(total=len(dict_aux), desc='Tabela:' , unit='Tabelas',color='green')
pbar.refresh()
for key, value in dict_aux.items():       
    con.execute('DROP TABLE IF EXISTS ' + value[0]) 
    con.execute(value[1])
    
    lista = [l for l in filtered if key in l]
    pbar1 = enlighten.Counter(total=len(lista), desc='Aquivo CSV: ' + value[0] , unit='Arquivos', color='yellow')
    pbar1.refresh()
    for item in lista:
        status_bar.update('Convertendo o arquivo: ' + str(item))
        df = pd.read_csv("zips/" + item, 
                         compression='zip',
                         header=None, 
                         sep=';', 
                         quotechar='"', 
                         encoding = "ISO-8859-1", 
                         low_memory=False,
                         dtype = value[3],
                         names=value[2])

        df.to_sql(name = value[0], con = connection, if_exists = 'append', index = False)
        pbar1.update()
    pbar.update()
    status_bar.update('Conversão Concluida!')
manager.stop()

In [ ]:
# criar os indices corretos 
list_sql = [
    """CREATE INDEX IF NOT EXISTS IDX_SOCIOS ON SOCIOS (CNPJ ASC)""",
    """CREATE INDEX IF NOT EXISTS IDX_ESTABELECIMENTOS ON ESTABELECIMENTOS (CNPJ ASC)""",
    """CREATE INDEX IF NOT EXISTS IDX_EMPRESAS ON EMPRESAS (CNPJ ASC)""",
    """ALTER TABLE ESTABELECIMENTOS ADD COLUMN CNPJ_FULL VARCHAR (14)""",
    """UPDATE ESTABELECIMENTOS set CNPJ_FULL = cnpj || cnpj_ordem || cnpj_dv""",
    """CREATE INDEX IF NOT EXISTS IDX_ESTABELECIMENTOS_CNPJ_FULL ON ESTABELECIMENTOS (CNPJ_FULL ASC)"""
    
]
list_sql[1]


In [ ]:
%%time
for i in list_sql:
    print(i)
    con.executescript(i)

In [ ]:
%%time
df = pd.read_sql_query("SELECT count(CNPJ) from ESTABELECIMENTOS", connection)
con.close()
df.head()

In [ ]:
2 ** 20

In [ ]:
1024 *1024

In [ ]:
%time
list_estabelecimentos = [
"cnpj", 
"cnpj_ordem", 
"cnpj_dv", 
"matriz_filial",
"nome",
"situacao", 
"data_situacao", 
"motivo_situacao", 
"cidade_exterior", 
"pais", 
"inicio_atividade", 
"cnae_fiscal", 
"cnae_secundario", 
"tipo_logradouro", 
"logradouro", 
"numero", 
"complemento", 
"bairro", 
"cep", 
"uf", 
"municipio", 
"ddd_1", 
"telefone_1", 
"ddd_2", 
"telefone_2", 
"ddd_fax", 
"numero_fax", 
"email", 
"situacao_especial", 
"data_situacao_especial"
]

In [ ]:
%time

import sqlite3

connection = sqlite3.connect(PATH_BD)
con = connection.cursor()

sql = """CREATE TABLE EMPRESAS (
    CNPJ            VARCHAR (8),
    RAZAO           VARCHAR,
    NATUREZA        INTEGER,
    QUALIFICACAO_PF INTEGER,
    CAPITAL         FLOAT,
    PORTE           INTEGER,
    ENTE_FEDERATIVO INTEGER
)
                """


con.execute('DROP TABLE IF EXISTS EMPRESAS') 
con.execute('VACUUM') 
con.execute(sql)


In [ ]:
lista = [l for l in filtered if "ESTABELE" in l]

pbar = enlighten.Counter(total=len(lista), desc='Aquivo:' , unit='Arquivos')
manager = enlighten.get_manager()
status_bar = manager.status_bar('Convertendo...',
                                color='blue',
                                justify=enlighten.Justify.CENTER)
for item in lista:
    status_bar.update('Convertendo o arquivo: ' + str(item))
    df = pd.read_csv("zips/" + item, 
                     compression='zip',
                     header=None, 
                     sep=';', 
                     quotechar='"', 
                     encoding = "ISO-8859-1", 
                     low_memory=False,
                     dtype = {'cnpj': str, 'cnpj_ordem': str, 'cnpj_dv': str},
                     names=list_estabelecimentos,
                     parse_dates=['data_situacao','inicio_atividade','data_situacao_especial'])


#    df['data_situacao']= df['data_situacao'].replace(0, np.nan)

#    df['data_situacao'] = pd.to_datetime(df['data_situacao'], format=formato_data)
#    df['inicio_atividade'] = pd.to_datetime(df['inicio_atividade'], format=formato_data)
#    df['data_situacao_especial'] = pd.to_datetime(df['data_situacao_especial'], format=formato_data)

    df.to_sql(name = 'ESTABELECIMENTOS', con = connection, if_exists = 'append', index = False)
    pbar.update()
status_bar.update('Conversão Concluida!' + str(item))

In [ ]:
list_empresas = [
"CNPJ", 
"RAZAO", 
"NATUREZA", 
"QUALIFICACAO_PF",
"CAPITAL",
"PORTE",
"ENTE_FEDERATIVO"
]

In [ ]:
lista = [l for l in filtered if "EMPRECSV" in l]

pbar = enlighten.Counter(total=len(lista), desc='Aquivo:' , unit='Arquivos')
manager = enlighten.get_manager()
status_bar = manager.status_bar('Convertendo...',
                                color='blue',
                                justify=enlighten.Justify.CENTER)
for item in lista:
    status_bar.update('Convertendo o arquivo: ' + str(item))
    df = pd.read_csv("zips/" + item, 
                     compression='zip',
                     header=None, 
                     sep=';', 
                     quotechar='"', 
                     encoding = "ISO-8859-1", 
                     low_memory=False,
                     dtype = {'cnpj': str},
                     names=list_empresas)

    df.to_sql(name = 'EMPRESAS', con = connection, if_exists = 'append', index = False)
    pbar.update()
status_bar.update('Conversão Concluida!')

In [ ]:


%time

import sqlite3

connection = sqlite3.connect(PATH_BD)
con = connection.cursor()

sql = """CREATE TABLE QUALIFICACOES (
                CODIGO    INTEGER NOT NULL,
                DESCRICAO VARCHAR
            )
                """


con.execute('DROP TABLE IF EXISTS QUALIFICACOES') 
con.execute('VACUUM') 
con.execute(sql)

In [ ]:
list_qualificacoes = [
"CODIGO", 
"DESCRICAO"
]

In [ ]:
lista = [l for l in filtered if "QUALSCSV" in l]


pbar = enlighten.Counter(total=len(lista), desc='Aquivo:' , unit='Arquivos')
manager = enlighten.get_manager()
status_bar = manager.status_bar('Convertendo...',
                                color='blue',
                                justify=enlighten.Justify.CENTER)
for item in lista:
    status_bar.update('Convertendo o arquivo: ' + str(item))
    df = pd.read_csv("zips/" + item, 
                     compression='zip',
                     header=None, 
                     sep=';', 
                     quotechar='"', 
                     encoding = "ISO-8859-1", 
                     low_memory=False,
                     dtype = {'cnpj': str},
                     names=list_qualificacoes)

    df.to_sql(name = 'QUALIFICACOES', con = connection, if_exists = 'append', index = False)
    pbar.update()
status_bar.update('Conversão Concluida!' + str(item))

In [ ]:
df_aux = pd.read_excel("auxiliar.xlsx")


In [ ]:
df_aux

In [ ]:
for index, row in df_aux.iterrows():
    print(index, row['titulo'], row['campos_panda'])


In [ ]:
df_aux.set_index('titulo').T.to_dict('list')

In [ ]:
dicionario = {
    'SIMPLES': [
        'SIMPLES',
        'CREATE TABLE IF NOT EXISTS DADOS_SIMPLES (\n    ID                 INTEGER     NOT NULL,\n    CNPJ               VARCHAR (8),\n    OPCAO_SIMPLES      VARCHAR (1),\n    DATA_OPCAO_SIMPLES DATE,\n    DATA_EXCLUSAO      DATE,\n    OPCAO_MEI          VARCHAR (1),\n    DATA_OPCAO_MEI     DATE,\n    DATA_EXCLUSAO_MEI  DATE\n)',
        ['cnpj','opcao_simples','data_opcao_simples','data_exclusao','opcao_mei','data_opcao_mei','data_exclusao_mei']
        ],
    'CNAECSV': [
        'CNAES',
        'CREATE TABLE IF NOT EXISTS CNAES (\n    CODIGO    VARCHAR NOT NULL,\n    DESCRICAO VARCHAR\n)',
        ['codigo','descricao']
        ],
     'MOTICSV': [
        'MOTIVOS',
        'CREATE TABLE IF NOT EXISTS MOTIVOS (\n    CODIGO    INTEGER NOT NULL,\n    DESCRICAO VARCHAR\n)',
        ['codigo','descricao']
        ],
     'MUNICCSV': [
        'MUNICIPIOS',
        'CREATE TABLE IF NOT EXISTS MUNICIPIOS (\n    CODIGO    INTEGER NOT NULL,\n    DESCRICAO VARCHAR\n)',
        ['codigo','descricao']
        ],
     'NATJUCSV': [
        'NATUREZAS',
        'CREATE TABLE IF NOT EXISTS NATUREZAS (\n    CODIGO    INTEGER NOT NULL,\n    DESCRICAO VARCHAR\n)',
        ['codigo','descricao']
        ],
     'PAISCSV': [
        'PAISES',
        'CREATE TABLE IF NOT EXISTS PAISES (\n    CODIGO    INTEGER NOT NULL,\n    DESCRICAO VARCHAR\n)',
        ['codigo','descricao']
        ],
     'QUALSCSV': [
        'QUALIFICACOES',
        'CREATE TABLE IF NOT EXISTS QUALIFICACOES (\n    CODIGO    INTEGER NOT NULL,\n    DESCRICAO VARCHAR\n)',
        ['codigo','descricao']
        ],
     'EMPRECSV': [
        'EMPRESAS',
        'CREATE TABLE IF NOT EXISTS EMPRESAS (\n    ID              INTEGER     NOT NULL,\n    CNPJ            VARCHAR (8),\n    RAZAO           VARCHAR,\n    NATUREZA        INTEGER,\n    QUALIFICACAO_PF INTEGER,\n    CAPITAL         FLOAT,\n    PORTE           INTEGER,\n    ENTE_FEDERATIVO INTEGER\n)',
        ['cnpj','razao','natureza','qualificacao_pf','capital','porte','ente_federativo']
        ],
    'ESTABELE': [
        'ESTABELECIMENTOS',
        'CREATE TABLE IF NOT EXISTS ESTABELECIMENTOS \n                    (CNPJ                   VARCHAR (8),\n                    CNPJ_ORDEM             VARCHAR (4),\n                    CNPJ_DV                VARCHAR (2),\n                    MATRIZ_FILIAL          INTEGER,\n                    NOME                   VARCHAR,\n                    SITUACAO               INTEGER,\n                    DATA_SITUACAO          DATE,\n                    MOTIVO_SITUACAO        VARCHAR,\n                    CIDADE_EXTERIOR        VARCHAR,\n                    PAIS                   INTEGER,\n                    INICIO_ATIVIDADE       DATE,\n                    CNAE_FISCAL            VARCHAR,\n                    CNAE_SECUNDARIO        VARCHAR,\n                    TIPO_LOGRADOURO        VARCHAR,\n                    LOGRADOURO             VARCHAR,\n                    NUMERO                 VARCHAR,\n                    COMPLEMENTO            VARCHAR,\n                    BAIRRO                 VARCHAR,\n                    CEP                    VARCHAR (8),\n                    UF                     VARCHAR (2),\n                    MUNICIPIO              INTEGER,\n                    DDD_1                  VARCHAR,\n                    TELEFONE_1             VARCHAR (11),\n                    DDD_2                  INTEGER,\n                    TELEFONE_2             VARCHAR (11),\n                    DDD_FAX                INTEGER,\n                    NUMERO_FAX             VARCHAR (11),\n                    EMAIL                  VARCHAR,\n                    SITUACAO_ESPECIAL      VARCHAR,\n                    DATA_SITUACAO_ESPECIAL DATE \n                    )',
        ['cnpj','ncnpj_ordem','ncnpj_dv','nmatriz_filial','nome','situacao','data_situacao','motivo_situacao','cidade_exterior','pais','inicio_atividade','cnae_fiscal','cnae_secundario','tipo_logradouro','logradouro','numero','complemento','bairro','cep','uf','municipio','ddd_1','telefone_1','ddd_2','telefone_2','ddd_fax','numero_fax','email','situacao_especial','data_situacao_especial']
        ],
    'SOCIOCSV': [
        'SOCIOS',
        'CREATE TABLE IF NOT EXISTS SOCIOS (\n    ID                               INTEGER      NOT NULL,\n    CNPJ                             VARCHAR (8),\n    IDENTIFICADOR_SOCIO              INTEGER,\n    NOME                             VARCHAR,\n    CPF_CNPJ                         VARCHAR (14),\n    QUALIFICACAO                     INTEGER,\n    DATA_ENTRADA_SOCIEDADE           DATE,\n    CODIGO_PAIS                      INTEGER,\n    CPF_REPRESENTANTE_LEGAL          VARCHAR (11),\n    NOME_REPRESENTANTE_LEGAL         VARCHAR,\n    QUALIFICACAO_REPRESENTANTE_LEGAL INTEGER,\n    FAIXA_ETARIA                     INTEGER\n)',
        ['cnpj','identificador_socio','nome','cpf_cnpj','qualificacao','data_entrada_sociedade','codigo_pais','cpf_representante_legal','nome_representante_legal','qualificacao_representante_legal','faixa_etaria']
        ]    
    }

In [ ]:
dicionario = {
    'SIMPLES': {
        'nome_tabela': 'SIMPLES',
        'create_tabela': 'CREATE TABLE IF NOT EXISTS DADOS_SIMPLES (\n    ID                 INTEGER     NOT NULL,\n    CNPJ               VARCHAR (8),\n    OPCAO_SIMPLES      VARCHAR (1),\n    DATA_OPCAO_SIMPLES DATE,\n    DATA_EXCLUSAO      DATE,\n    OPCAO_MEI          VARCHAR (1),\n    DATA_OPCAO_MEI     DATE,\n    DATA_EXCLUSAO_MEI  DATE\n)',
        'campos_panda': ['cnpj','opcao_simples','data_opcao_simples','data_exclusao','opcao_mei','data_opcao_mei','data_exclusao_mei']
        },
     'CNAECSV': {
        'nome_tabela': 'CNAES',
        'create_tabela': 'CREATE TABLE IF NOT EXISTS CNAES (\n    CODIGO    VARCHAR NOT NULL,\n    DESCRICAO VARCHAR\n)',
        'campos_panda': ['codigo','descricao']
         },
     'MOTICSV': {
        'nome_tabela': 'MOTIVOS',
        'create_tabela': 'CREATE TABLE IF NOT EXISTS MOTIVOS (\n    CODIGO    INTEGER NOT NULL,\n    DESCRICAO VARCHAR\n)',
        'campos_panda': ['codigo','descricao']
         },
     'MUNICCSV': {
        'nome_tabela': 'MUNICIPIOS',
        'create_tabela': 'CREATE TABLE IF NOT EXISTS MUNICIPIOS (\n    CODIGO    INTEGER NOT NULL,\n    DESCRICAO VARCHAR\n)',
        'campos_panda': ['codigo','descricao']
         },
     'NATJUCSV': {
        'nome_tabela': 'NATUREZAS',
        'create_tabela': 'CREATE TABLE IF NOT EXISTS NATUREZAS (\n    CODIGO    INTEGER NOT NULL,\n    DESCRICAO VARCHAR\n)',
        'campos_panda': ['codigo','descricao']
         },
    'PAISCSV': {
        'nome_tabela': 'PAISES',
        'create_tabela': 'CREATE TABLE IF NOT EXISTS PAISES (\n    CODIGO    INTEGER NOT NULL,\n    DESCRICAO VARCHAR\n)',
        'campos_panda': ['codigo','descricao']
        },
     'QUALSCSV': {
        'nome_tabela': 'QUALIFICACOES',
        'create_tabela': 'CREATE TABLE IF NOT EXISTS QUALIFICACOES (\n    CODIGO    INTEGER NOT NULL,\n    DESCRICAO VARCHAR\n)',
        'campos_panda': ['codigo','descricao']
         },
     'EMPRECSV': {
        'nome_tabela': 'EMPRESAS',
        'create_tabela': 'CREATE TABLE IF NOT EXISTS EMPRESAS (\n    ID              INTEGER     NOT NULL,\n    CNPJ            VARCHAR (8),\n    RAZAO           VARCHAR,\n    NATUREZA        INTEGER,\n    QUALIFICACAO_PF INTEGER,\n    CAPITAL         FLOAT,\n    PORTE           INTEGER,\n    ENTE_FEDERATIVO INTEGER\n)',
        'campos_panda': ['cnpj','razao','natureza','qualificacao_pf','capital','porte','ente_federativo']
         },
     'ESTABELE': {
        'nome_tabela': 'ESTABELECIMENTOS',
        'create_tabela': 'CREATE TABLE IF NOT EXISTS ESTABELECIMENTOS \n                    (CNPJ                   VARCHAR (8),\n                    CNPJ_ORDEM             VARCHAR (4),\n                    CNPJ_DV                VARCHAR (2),\n                    MATRIZ_FILIAL          INTEGER,\n                    NOME                   VARCHAR,\n                    SITUACAO               INTEGER,\n                    DATA_SITUACAO          DATE,\n                    MOTIVO_SITUACAO        VARCHAR,\n                    CIDADE_EXTERIOR        VARCHAR,\n                    PAIS                   INTEGER,\n                    INICIO_ATIVIDADE       DATE,\n                    CNAE_FISCAL            VARCHAR,\n                    CNAE_SECUNDARIO        VARCHAR,\n                    TIPO_LOGRADOURO        VARCHAR,\n                    LOGRADOURO             VARCHAR,\n                    NUMERO                 VARCHAR,\n                    COMPLEMENTO            VARCHAR,\n                    BAIRRO                 VARCHAR,\n                    CEP                    VARCHAR (8),\n                    UF                     VARCHAR (2),\n                    MUNICIPIO              INTEGER,\n                    DDD_1                  VARCHAR,\n                    TELEFONE_1             VARCHAR (11),\n                    DDD_2                  INTEGER,\n                    TELEFONE_2             VARCHAR (11),\n                    DDD_FAX                INTEGER,\n                    NUMERO_FAX             VARCHAR (11),\n                    EMAIL                  VARCHAR,\n                    SITUACAO_ESPECIAL      VARCHAR,\n                    DATA_SITUACAO_ESPECIAL DATE \n                    )',
        'campos_panda': ['cnpj','ncnpj_ordem','ncnpj_dv','nmatriz_filial','nome','situacao','data_situacao','motivo_situacao','cidade_exterior','pais','inicio_atividade','cnae_fiscal','cnae_secundario','tipo_logradouro','logradouro','numero','complemento','bairro','cep','uf','municipio','ddd_1','telefone_1','ddd_2','telefone_2','ddd_fax','numero_fax','email','situacao_especial','data_situacao_especial']
         },
     'SOCIOCSV': {
        'nome_tabela': 'SOCIOS',
        'create_tabela': 'CREATE TABLE IF NOT EXISTS SOCIOS (\n    ID                               INTEGER      NOT NULL,\n    CNPJ                             VARCHAR (8),\n    IDENTIFICADOR_SOCIO              INTEGER,\n    NOME                             VARCHAR,\n    CPF_CNPJ                         VARCHAR (14),\n    QUALIFICACAO                     INTEGER,\n    DATA_ENTRADA_SOCIEDADE           DATE,\n    CODIGO_PAIS                      INTEGER,\n    CPF_REPRESENTANTE_LEGAL          VARCHAR (11),\n    NOME_REPRESENTANTE_LEGAL         VARCHAR,\n    QUALIFICACAO_REPRESENTANTE_LEGAL INTEGER,\n    FAIXA_ETARIA                     INTEGER\n)',
        'campos_panda': ['cnpj','identificador_socio','nome','cpf_cnpj','qualificacao','data_entrada_sociedade','codigo_pais','cpf_representante_legal','nome_representante_legal','qualificacao_representante_legal','faixa_etaria']
         }
}

In [ ]:
dicionario
for key, value in dicionario.items():
    print(key)
    for i, val in enumerate(value):
        print (i, ",",val)

In [ ]:
dicionario['SIMPLES'][2]

In [ ]:
back = pd.read_sql('select * from estabelecimentos', connection)

back

In [ ]:
df = pd.DataFrame(filtered)
df[df[0].str.contains("EMPRECSV", regex=False)]


In [ ]:
lista = [l for l in filtered if "QUALSCSV" in l]
lista

In [ ]:
filtered

In [ ]:
list_tabelas = [
                "SIMPLES", 
                "CNAECSV",
                "MOTICSV",
                "MUNICCSV",
                "NATJUCSV",
                "PAISCSV",
                "QUALSCSV",
                "EMPRECSV",
                "ESTABELE",
                "SOCIOCSV",
                "EMPRECSV"]



In [ ]:
for l in list_tabelas:
    lista = [item for item in filtered if l in item]
    display(lista)

In [ ]:

CREATE TABLE IF NOT EXISTS ESTABELECIMENTOS 
                    (CNPJ                   VARCHAR (8),
                    CNPJ_ORDEM             VARCHAR (4),
                    CNPJ_DV                VARCHAR (2),
                    MATRIZ_FILIAL          INTEGER,
                    NOME                   VARCHAR,
                    SITUACAO               INTEGER,
                    DATA_SITUACAO          DATE,
                    MOTIVO_SITUACAO        VARCHAR,
                    CIDADE_EXTERIOR        VARCHAR,
                    PAIS                   INTEGER,
                    INICIO_ATIVIDADE       DATE,
                    CNAE_FISCAL            VARCHAR,
                    CNAE_SECUNDARIO        VARCHAR,
                    TIPO_LOGRADOURO        VARCHAR,
                    LOGRADOURO             VARCHAR,
                    NUMERO                 VARCHAR,
                    COMPLEMENTO            VARCHAR,
                    BAIRRO                 VARCHAR,
                    CEP                    VARCHAR (8),
                    UF                     VARCHAR (2),
                    MUNICIPIO              INTEGER,
                    DDD_1                  VARCHAR,
                    TELEFONE_1             VARCHAR (11),
                    DDD_2                  INTEGER,
                    TELEFONE_2             VARCHAR (11),
                    DDD_FAX                INTEGER,
                    NUMERO_FAX             VARCHAR (11),
                    EMAIL                  VARCHAR,
                    SITUACAO_ESPECIAL      VARCHAR,
                    DATA_SITUACAO_ESPECIAL DATE 
                    )

CREATE TABLE IF NOT EXISTS EMPRESAS (
    ID              INTEGER     NOT NULL,
    CNPJ            VARCHAR (8),
    RAZAO           VARCHAR,
    NATUREZA        INTEGER,
    QUALIFICACAO_PF INTEGER,
    CAPITAL         FLOAT,
    PORTE           INTEGER,
    ENTE_FEDERATIVO INTEGER
)

CREATE TABLE IF NOT EXISTS CNAES (
    CODIGO    VARCHAR NOT NULL,
    DESCRICAO VARCHAR
)
CREATE TABLE IF NOT EXISTS DADOS_SIMPLES (
    ID                 INTEGER     NOT NULL,
    CNPJ               VARCHAR (8),
    OPCAO_SIMPLES      VARCHAR (1),
    DATA_OPCAO_SIMPLES DATE,
    DATA_EXCLUSAO      DATE,
    OPCAO_MEI          VARCHAR (1),
    DATA_OPCAO_MEI     DATE,
    DATA_EXCLUSAO_MEI  DATE
)
CREATE TABLE IF NOT EXISTS MUNICIPIOS (
    CODIGO    INTEGER NOT NULL,
    DESCRICAO VARCHAR
)
CREATE TABLE IF NOT EXISTS NATUREZAS (
    CODIGO    INTEGER NOT NULL,
    DESCRICAO VARCHAR
)
CREATE TABLE IF NOT EXISTS PAISES (
    CODIGO    INTEGER NOT NULL,
    DESCRICAO VARCHAR
)
CREATE TABLE IF NOT EXISTS QUALIFICACOES (
    CODIGO    INTEGER NOT NULL,
    DESCRICAO VARCHAR
)
CREATE TABLE IF NOT EXISTS SOCIOS (
    ID                               INTEGER      NOT NULL,
    CNPJ                             VARCHAR (8),
    IDENTIFICADOR_SOCIO              INTEGER,
    NOME                             VARCHAR,
    CPF_CNPJ                         VARCHAR (14),
    QUALIFICACAO                     INTEGER,
    DATA_ENTRADA_SOCIEDADE           DATE,
    CODIGO_PAIS                      INTEGER,
    CPF_REPRESENTANTE_LEGAL          VARCHAR (11),
    NOME_REPRESENTANTE_LEGAL         VARCHAR,
    QUALIFICACAO_REPRESENTANTE_LEGAL INTEGER,
    FAIXA_ETARIA                     INTEGER
)


In [ ]:

dtypes = np.dtype(
    [
        ("a", str),
        ("b", int),
        ("c", float),
        ("d", np.datetime64),
        
    ]
)
df = pd.DataFrame(np.empty(0, dtype=dtypes))

In [ ]:
encoding_list = ['ascii', 'big5', 'big5hkscs', 'cp037', 'cp273', 'cp424', 'cp437', 'cp500', 'cp720', 'cp737'
                 , 'cp775', 'cp850', 'cp852', 'cp855', 'cp856', 'cp857', 'cp858', 'cp860', 'cp861', 'cp862'
                 , 'cp863', 'cp864', 'cp865', 'cp866', 'cp869', 'cp874', 'cp875', 'cp932', 'cp949', 'cp950'
                 , 'cp1006', 'cp1026', 'cp1125', 'cp1140', 'cp1250', 'cp1251', 'cp1252', 'cp1253', 'cp1254'
                 , 'cp1255', 'cp1256', 'cp1257', 'cp1258', 'euc_jp', 'euc_jis_2004', 'euc_jisx0213', 'euc_kr'
                 , 'gb2312', 'gbk', 'gb18030', 'hz', 'iso2022_jp', 'iso2022_jp_1', 'iso2022_jp_2'
                 , 'iso2022_jp_2004', 'iso2022_jp_3', 'iso2022_jp_ext', 'iso2022_kr', 'latin_1', 'iso8859_2'
                 , 'iso8859_3', 'iso8859_4', 'iso8859_5', 'iso8859_6', 'iso8859_7', 'iso8859_8', 'iso8859_9'
                 , 'iso8859_10', 'iso8859_11', 'iso8859_13', 'iso8859_14', 'iso8859_15', 'iso8859_16', 'johab'
                 , 'koi8_r', 'koi8_t', 'koi8_u', 'kz1048', 'mac_cyrillic', 'mac_greek', 'mac_iceland', 'mac_latin2'
                 , 'mac_roman', 'mac_turkish', 'ptcp154', 'shift_jis', 'shift_jis_2004', 'shift_jisx0213', 'utf_32'
                 , 'utf_32_be', 'utf_32_le', 'utf_16', 'utf_16_be', 'utf_16_le', 'utf_7', 'utf_8', 'utf_8_sig']

for encoding in encoding_list:
    worked = True
    try:
        df = pd.read_csv('K3241.K03200Y0.D10710.ESTABELE.zip', compression='zip',header=None, sep=';', quotechar='"', encoding=encoding)
    except:
        worked = False
    if worked:
        print(encoding, ':\n', df.head())

In [ ]:
import time

manager = enlighten.get_manager()
status_bar = manager.status_bar('Static Message',
                                color='blue',
                                justify=enlighten.Justify.CENTER)
counter = enlighten.Counter(desc='Basic', unit='ticks')
for num in range(100):
    time.sleep(0.1)  # Simulate work
    status_bar.update('Updated static message' + str(num))
    counter.update()


In [ ]:
import math
import requests, enlighten

url = 'http://200.152.38.155/CNPJ/F.K03200$W.SIMPLES.CSV.D10710.zip'
fname = 'F.K03200$W.SIMPLES.CSV.D10710.zip'

# Should be one global variable
MANAGER = enlighten.get_manager()

r = requests.get(url, stream = True)
assert r.status_code == 200, r.status_code
dlen = int(r.headers.get('Content-Length', '0')) or None

with MANAGER.counter(color = 'green', total = dlen and math.ceil(dlen / 2 ** 20), unit = 'MiB', leave = False) as ctr, \
     open(fname, 'wb', buffering = 2 ** 24) as f:
    for chunk in r.iter_content(chunk_size = 2 ** 20):
        # print(chunk[-16:].hex().upper())
        f.write(chunk)
        ctr.update()

In [ ]:
list_urls = re.findall(r'href=[\'"]?([^\'" >]+)', requests.get(url).text)
filtered = list(filter(lambda url: url.split(".")[-1] in ["zip","pdf"], list_urls))
tamanho = len(filtered)
for indice, elemento in tqdm(enumerate(filtered)):
        resposta = requests.get(url + elemento, stream=True, headers={"Accept-Encoding": None})
        if resposta.status_code == requests.codes.ok:
            total_size = int(resposta.headers.get("Content-Length", 0))
            block_size = 1024

            print(f"Downloading {elemento}")

            with tqdm(total=total_size,file=sys.stdout) as pbar:
                with open(elemento, "wb") as f:
                    for data in resposta.iter_content(block_size):
                        pbar.set_description(f"Processing {indice +1}")
                        pbar.update(len(data))
                        f.write(data)
        else:
                resposta.raise_for_status()


In [ ]:
from tqdm import tqdm
from time import sleep

In [ ]:
pbar = tqdm(["a", "b", "c", "d"])
for char in pbar:
    sleep(0.25)
    pbar.set_description("Processing %s" % char)

In [ ]:
donloads(url, list_urls[1])

In [ ]:
def donloads(url, arquivo):
    resposta = requests.get(url + arquivo, stream=True, headers={"Accept-Encoding": None})
    if resposta.status_code == requests.codes.ok:
        total_size = int(resposta.headers.get("Content-Length", 0))
        block_size = 1024

        print(f"Downloading {arquivo}")

        t = tqdm(total=total_size, unit="iB", unit_scale=True)
        with open(arquivo, "wb") as f:
            for data in resposta.iter_content(block_size):
                t.update(len(data))
                f.write(data)
        t.close()
    else:
        resposta.raise_for_status()

In [ ]:

req = requests.get(url, stream=True, headers={"Accept-Encoding": None})

In [ ]:
req.links

In [ ]:
url = 'http://200.152.38.155/CNPJ/'

In [ ]:
def baixa_arquivo(url, endereco):
    # faz a requisição 
    resposta = requests.get(url)
    if resposta.status_code == requests.codes.ok:
        with open('endereco', 'wb')as novo_arquivo:
            novo_arquivo.write(resposta.content)
        print("Download finalizado, salvo em: ()".format(endereco))
    else:
        respósta.raise_for_status()

In [ ]:
baixa_arquivo(url, 'teste.html')

In [ ]:
data = str(urlopen(URL_BASE_RFB).read(), encoding='utf8')

    # Pega todas as urls
    urls = re.findall(r'href=[\'"]?([^\'" >]+)', data)

In [ ]:
resposta = requests.get(url).text

In [ ]:
print(resposta.text)

In [ ]:
import re

In [ ]:
list_urls = re.findall(r'href=[\'"]?([^\'" >]+)', requests.get(url).text)

In [ ]:
list_urls

In [ ]:
filtered = list(filter(lambda url: url.split(".")[-1] in ["zip","pdf"], list_urls))
filtered

In [ ]:
for indice, elemento in tqdm(enumerate(filtered)):
    print (f"{indice:<10}{elemento}")

In [ ]:
def donloads(url, arquivo):
    req = requests.get(url + arquivo, stream=True, headers={"Accept-Encoding": None})
    if resposta.status_code == requests.codes.ok:
        total_size = int(req.headers.get("Content-Length", 0))
        block_size = 1024

        print(f"Downloading {arquivo}")

        t = tqdm(total=total_size, unit="iB", unit_scale=True)
        with open(arquivo, "wb") as f:
            for data in req.iter_content(block_size):
                t.update(len(data))
                f.write(data)
        t.close()
    else:
        resposta.raise_for_status()

In [ ]:
for i in list_urls:
    if i.split(".")[-1] in ["zip","pdf"]:
        donloads(url, i)

In [ ]:
donloads(url, list_urls[1])

In [ ]:
list_urls

In [ ]:
filtered

In [ ]:
filtered[8]

In [ ]:
teste = pd.read_csv("zips/" + filtered[8], 
                         compression='zip',
                         header=None, 
                         sep=';', 
                         quotechar='"', 
                         encoding = "ISO-8859-1", 
                         low_memory=False,
                         dtype = value[3],
                         names=value[2])

In [ ]:
teste

In [ ]:
dict_aux